<a href="https://www.kaggle.com/code/trmnhviet/retrain-model?scriptVersionId=297286048" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import pandas as pd
import os
from sklearn.metrics import (
    accuracy_score, roc_auc_score, 
    precision_score, recall_score, f1_score,
    confusion_matrix
)
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ==================== SEED SETTING ====================
def set_seed(seed=42):
    """Set all random seeds for reproducibility"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)



In [2]:

class ChannelAttention(nn.Module):
    """CBAM Channel Attention Module"""
    def __init__(self, in_planes, ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)


class SpatialAttention(nn.Module):
    """CBAM Spatial Attention Module"""
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(out))


class CBAM(nn.Module):
    """Convolutional Block Attention Module"""
    def __init__(self, channels, ratio=16):
        super().__init__()
        self.channel_attention = ChannelAttention(channels, ratio)
        self.spatial_attention = SpatialAttention()
    
    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x


class CrossDiseaseAttention(nn.Module):
    """
    Cross-Disease Attention Module
    Uses features from one disease to enhance features of another
    """
    def __init__(self, channels, reduction=16):
        super().__init__()
        
        # Channel-wise cross attention
        self.channel_gate = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1, bias=False),
            nn.Sigmoid()
        )
        
        # Spatial cross attention
        self.spatial_gate = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.BatchNorm2d(channels // reduction),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, 1, 1, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, feat_main, feat_aux):
        """
        feat_main: main task features
        feat_aux: auxiliary task features (for attention)
        """
        # Channel attention from auxiliary features
        channel_att = self.channel_gate(feat_aux)
        feat = feat_main * channel_att
        
        # Spatial attention from auxiliary features
        spatial_att = self.spatial_gate(feat_aux)
        feat = feat * spatial_att
        
        return feat


class CANet(nn.Module):
    """
    Cross-disease Attention Network (CANet)
    Multi-task learning for DR and DME with cross-disease attention
    """
    def __init__(self, num_dr=5, num_dme=3, pretrained=True):
        super().__init__()
        
        # Backbone: ResNet50
        resnet = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        
        channels = 2048
        
        # Global pooling
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # Task-specific CBAM modules
        self.cbam_dr = CBAM(channels)
        self.cbam_dme = CBAM(channels)
        
        # Cross-disease attention modules (KEY INNOVATION)
        self.cross_dr_from_dme = CrossDiseaseAttention(channels)  # DME → DR
        self.cross_dme_from_dr = CrossDiseaseAttention(channels)  # DR → DME
        
        # Main classifiers
        self.fc_dr = nn.Linear(channels, num_dr)
        self.fc_dme = nn.Linear(channels, num_dme)
        
        # Auxiliary classifiers (for auxiliary loss)
        self.fc_dr_aux = nn.Linear(channels, num_dr)
        self.fc_dme_aux = nn.Linear(channels, num_dme)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize classifier weights"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Shared backbone features
        feat = self.backbone(x)  # B x 2048 x H x W
        
        # Task-specific features with CBAM
        feat_dr = self.cbam_dr(feat)
        feat_dme = self.cbam_dme(feat)
        
        # Cross-disease attention (CORE OF CANet)
        feat_dr_enhanced = self.cross_dr_from_dme(feat_dr, feat_dme)
        feat_dme_enhanced = self.cross_dme_from_dr(feat_dme, feat_dr)
        
        # Global pooling
        vec_dr_main = self.global_pool(feat_dr_enhanced).flatten(1)
        vec_dme_main = self.global_pool(feat_dme_enhanced).flatten(1)
        vec_dr_aux = self.global_pool(feat_dr).flatten(1)
        vec_dme_aux = self.global_pool(feat_dme).flatten(1)
        
        # Apply dropout
        vec_dr_main = self.dropout(vec_dr_main)
        vec_dme_main = self.dropout(vec_dme_main)
        vec_dr_aux = self.dropout(vec_dr_aux)
        vec_dme_aux = self.dropout(vec_dme_aux)
        
        # Classification
        out_dr = self.fc_dr(vec_dr_main)
        out_dme = self.fc_dme(vec_dme_main)
        out_dr_aux = self.fc_dr_aux(vec_dr_aux)
        out_dme_aux = self.fc_dme_aux(vec_dme_aux)
        
        return out_dr, out_dme, out_dr_aux, out_dme_aux


In [3]:
class IDRiDMultitaskDataset(Dataset):
    """IDRiD Dataset for multitask learning (DR + DME)"""
    
    def __init__(self, image_dir, labels_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        
        # Load labels
        self.data = pd.read_csv(labels_file)
        
        self.image_names = self.data['Image name'].values
        self.labels_dr = self.data['Retinopathy grade'].values
        self.labels_dme = self.data['Risk of macular edema '].values
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        # Load image
        img_name = self.image_names[idx]
        img_path = os.path.join(self.image_dir, f"{img_name}.jpg")
        
        # Try alternative extensions
        if not os.path.exists(img_path):
            for ext in ['.png', '.jpeg', '.JPG']:
                alt_path = os.path.join(self.image_dir, f"{img_name}{ext}")
                if os.path.exists(alt_path):
                    img_path = alt_path
                    break
        
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label_dr = self.labels_dr[idx]
        label_dme = self.labels_dme[idx]
        
        return image, (label_dr, label_dme), img_name

In [4]:
def get_transforms(train=True, img_size=224):
    """Advanced data augmentation for retinal images"""
    
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
    
    if train:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=30),
            transforms.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.1
            ),
            transforms.RandomAffine(
                degrees=0,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1)
            ),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            normalize,
        ])


In [5]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def calculate_metrics(y_true, y_pred, y_prob, num_classes):
    """Calculate comprehensive metrics"""
    
    acc = accuracy_score(y_true, y_pred)
    
    # AUC
    if num_classes == 2:
        auc = roc_auc_score(y_true, y_prob[:, 1])
    else:
        try:
            y_true_bin = label_binarize(y_true, classes=list(range(num_classes)))
            auc = roc_auc_score(y_true_bin, y_prob, average='macro', multi_class='ovr')
        except:
            auc = 0.0
    
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    # Confusion matrix metrics (for binary or first class)
    cm = confusion_matrix(y_true, y_pred)
    if num_classes == 2:
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else:
        # For multi-class, calculate for "any disease" vs "no disease"
        binary_true = (y_true > 0).astype(int)
        binary_pred = (y_pred > 0).astype(int)
        cm_bin = confusion_matrix(binary_true, binary_pred)
        if cm_bin.shape == (2, 2):
            tn, fp, fn, tp = cm_bin.ravel()
            sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
            specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        else:
            sensitivity, specificity = 0.0, 0.0
    
    return {
        'accuracy': acc,
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'sensitivity': sensitivity,
        'specificity': specificity
    }

In [6]:

def train_epoch(model, dataloader, criterion, optimizer, device, lambda_aux=0.25):
    """Train for one epoch with multi-task loss"""
    
    model.train()
    losses = AverageMeter()
    losses_dr_main = AverageMeter()
    losses_dme_main = AverageMeter()
    losses_dr_aux = AverageMeter()
    losses_dme_aux = AverageMeter()
    
    pbar = tqdm(dataloader, desc='Training')
    for images, (labels_dr, labels_dme), _ in pbar:
        images = images.to(device)
        labels_dr = labels_dr.to(device)
        labels_dme = labels_dme.to(device)
        
        # Forward pass
        out_dr, out_dme, out_dr_aux, out_dme_aux = model(images)
        
        # Multi-task loss
        loss_dr_main = criterion(out_dr, labels_dr)
        loss_dme_main = criterion(out_dme, labels_dme)
        loss_dr_aux = criterion(out_dr_aux, labels_dr)
        loss_dme_aux = criterion(out_dme_aux, labels_dme)
        
        # Combined loss (main losses + weighted auxiliary losses)
        loss = (loss_dr_main + loss_dme_main + 
                lambda_aux * (loss_dr_aux + loss_dme_aux))
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # Update metrics
        losses.update(loss.item(), images.size(0))
        losses_dr_main.update(loss_dr_main.item(), images.size(0))
        losses_dme_main.update(loss_dme_main.item(), images.size(0))
        losses_dr_aux.update(loss_dr_aux.item(), images.size(0))
        losses_dme_aux.update(loss_dme_aux.item(), images.size(0))
        
        pbar.set_postfix({
            'loss': f'{losses.avg:.4f}',
            'dr': f'{losses_dr_main.avg:.4f}',
            'dme': f'{losses_dme_main.avg:.4f}'
        })
    
    return {
        'total': losses.avg,
        'dr_main': losses_dr_main.avg,
        'dme_main': losses_dme_main.avg,
        'dr_aux': losses_dr_aux.avg,
        'dme_aux': losses_dme_aux.avg
    }


def validate_multitask(model, dataloader, criterion, device, lambda_aux=0.25):
    """Validate multitask model"""
    
    model.eval()
    losses = AverageMeter()
    
    all_dr_labels, all_dme_labels = [], []
    all_dr_preds, all_dme_preds = [], []
    all_dr_probs, all_dme_probs = [], []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for images, (labels_dr, labels_dme), _ in pbar:
            images = images.to(device)
            labels_dr_cpu = labels_dr.numpy()
            labels_dme_cpu = labels_dme.numpy()
            labels_dr = labels_dr.to(device)
            labels_dme = labels_dme.to(device)
            
            # Forward pass
            out_dr, out_dme, out_dr_aux, out_dme_aux = model(images)
            
            # Loss
            loss_dr_main = criterion(out_dr, labels_dr)
            loss_dme_main = criterion(out_dme, labels_dme)
            loss_dr_aux = criterion(out_dr_aux, labels_dr)
            loss_dme_aux = criterion(out_dme_aux, labels_dme)
            
            loss = (loss_dr_main + loss_dme_main + 
                    lambda_aux * (loss_dr_aux + loss_dme_aux))
            
            # Predictions (use main outputs only)
            probs_dr = torch.softmax(out_dr, dim=1)
            probs_dme = torch.softmax(out_dme, dim=1)
            preds_dr = torch.argmax(probs_dr, dim=1)
            preds_dme = torch.argmax(probs_dme, dim=1)
            
            # Store results
            all_dr_labels.extend(labels_dr_cpu)
            all_dme_labels.extend(labels_dme_cpu)
            all_dr_preds.extend(preds_dr.cpu().numpy())
            all_dme_preds.extend(preds_dme.cpu().numpy())
            all_dr_probs.extend(probs_dr.cpu().numpy())
            all_dme_probs.extend(probs_dme.cpu().numpy())
            
            losses.update(loss.item(), images.size(0))
            pbar.set_postfix({'loss': f'{losses.avg:.4f}'})
    
    # Convert to arrays
    all_dr_labels = np.array(all_dr_labels)
    all_dme_labels = np.array(all_dme_labels)
    all_dr_preds = np.array(all_dr_preds)
    all_dme_preds = np.array(all_dme_preds)
    all_dr_probs = np.array(all_dr_probs)
    all_dme_probs = np.array(all_dme_probs)
    
    # Calculate metrics for each task
    metrics_dr = calculate_metrics(all_dr_labels, all_dr_preds, all_dr_probs, num_classes=5)
    metrics_dme = calculate_metrics(all_dme_labels, all_dme_preds, all_dme_probs, num_classes=3)
    
    # Joint accuracy (both tasks correct)
    joint_correct = np.sum(
        (all_dr_preds == all_dr_labels) & (all_dme_preds == all_dme_labels)
    )
    joint_acc = joint_correct / len(all_dr_labels)
    
    return {
        'loss': losses.avg,
        'dr': metrics_dr,
        'dme': metrics_dme,
        'joint_accuracy': joint_acc
    }


In [7]:
def train_canet(
    train_loader,
    val_loader,
    epochs=100,
    lr=0.001,
    weight_decay=1e-4,
    lambda_aux=0.25,
    device='cuda',
    save_dir='models'
):
    """Main training function for CANet"""
    
    # Create model
    model = CANet(num_dr=5, num_dme=3, pretrained=True)
    model = model.to(device)
    
    print(f'Total parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M')
    print(f'Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6:.2f}M')
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer - different LR for backbone and heads
    backbone_params = []
    head_params = []
    
    for name, param in model.named_parameters():
        if 'backbone' in name:
            backbone_params.append(param)
        else:
            head_params.append(param)
    
    optimizer = optim.Adam([
        {'params': backbone_params, 'lr': lr * 0.1},  # Lower LR for pretrained backbone
        {'params': head_params, 'lr': lr}
    ], weight_decay=weight_decay)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, 
        T_0=10,  # Restart every 10 epochs
        T_mult=2,
        eta_min=1e-6
    )
    
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_dr_acc': [],
        'val_dme_acc': [],
        'val_joint_acc': [],
        'val_dr_auc': [],
        'val_dme_auc': []
    }
    
    best_joint_acc = 0.0
    best_dr_auc = 0.0
    best_dme_auc = 0.0
    
    os.makedirs(save_dir, exist_ok=True)
    
    # Training loop
    for epoch in range(epochs):
        print(f'\n{"="*70}')
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'{"="*70}')
        
        # Train
        train_losses = train_epoch(
            model, train_loader, criterion, optimizer, device, lambda_aux
        )
        
        # Validate
        val_results = validate_multitask(
            model, val_loader, criterion, device, lambda_aux
        )
        
        # Update scheduler
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        
        # Store history
        history['train_loss'].append(train_losses['total'])
        history['val_loss'].append(val_results['loss'])
        history['val_dr_acc'].append(val_results['dr']['accuracy'])
        history['val_dme_acc'].append(val_results['dme']['accuracy'])
        history['val_joint_acc'].append(val_results['joint_accuracy'])
        history['val_dr_auc'].append(val_results['dr']['auc'])
        history['val_dme_auc'].append(val_results['dme']['auc'])
        
        # Print metrics
        print(f'\nLearning Rate: {current_lr:.6f}')
        print(f'\nTrain Loss: {train_losses["total"]:.4f}')
        print(f'  - DR Main: {train_losses["dr_main"]:.4f}')
        print(f'  - DME Main: {train_losses["dme_main"]:.4f}')
        print(f'  - DR Aux: {train_losses["dr_aux"]:.4f}')
        print(f'  - DME Aux: {train_losses["dme_aux"]:.4f}')
        
        print(f'\nValidation Loss: {val_results["loss"]:.4f}')
        
        print(f'\nDR Metrics:')
        for k, v in val_results['dr'].items():
            print(f'  {k}: {v:.4f}')
        
        print(f'\nDME Metrics:')
        for k, v in val_results['dme'].items():
            print(f'  {k}: {v:.4f}')
        
        print(f'\nJoint Accuracy: {val_results["joint_accuracy"]:.4f}')
        
        # Save best models
        if val_results['joint_accuracy'] > best_joint_acc:
            best_joint_acc = val_results['joint_accuracy']
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'joint_acc': best_joint_acc,
                'dr_metrics': val_results['dr'],
                'dme_metrics': val_results['dme']
            }, os.path.join(save_dir, 'best_joint_acc.pth'))
            print(f'\n✓ Saved best joint accuracy model: {best_joint_acc:.4f}')
        
        if val_results['dr']['auc'] > best_dr_auc:
            best_dr_auc = val_results['dr']['auc']
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'dr_auc': best_dr_auc
            }, os.path.join(save_dir, 'best_dr_auc.pth'))
            print(f'✓ Saved best DR AUC model: {best_dr_auc:.4f}')
        
        if val_results['dme']['auc'] > best_dme_auc:
            best_dme_auc = val_results['dme']['auc']
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'dme_auc': best_dme_auc
            }, os.path.join(save_dir, 'best_dme_auc.pth'))
            print(f'Saved best DME AUC model: {best_dme_auc:.4f}')
    
    print(f'\n{"="*70}')
    print('Training completed!')
    print(f'Best Joint Accuracy: {best_joint_acc:.4f}')
    print(f'Best DR AUC: {best_dr_auc:.4f}')
    print(f'Best DME AUC: {best_dme_auc:.4f}')
    print(f'{"="*70}')
    
    return model, history


In [8]:

def main():
    """Main execution"""
    
    # Configuration
    config = {
        'batch_size': 16,
        'epochs': 100,
        'lr': 0.001,
        'weight_decay': 1e-4,
        'lambda_aux': 0.25,  # Weight for auxiliary losses
        'img_size': 224,
        'num_workers': 4,
    }
    
    # Paths - UPDATE THESE!
    TRAIN_IMG_DIR = '/kaggle/input/idrid-dataset/B. Disease Grading/B. Disease Grading/1. Original Images/a. Training Set'
    TRAIN_LABELS = '/kaggle/input/idrid-dataset/B. Disease Grading/B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv'
    VAL_IMG_DIR = '/kaggle/input/idrid-dataset/B. Disease Grading/B. Disease Grading/1. Original Images/b. Testing Set'
    VAL_LABELS = '/kaggle/input/idrid-dataset/B. Disease Grading/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv'
    
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # Transforms
    train_transform = get_transforms(train=True, img_size=config['img_size'])
    val_transform = get_transforms(train=False, img_size=config['img_size'])
    
    # Datasets
    train_dataset = IDRiDMultitaskDataset(
        TRAIN_IMG_DIR, TRAIN_LABELS, transform=train_transform
    )
    val_dataset = IDRiDMultitaskDataset(
        VAL_IMG_DIR, VAL_LABELS, transform=val_transform
    )
    
    print(f'Train samples: {len(train_dataset)}')
    print(f'Val samples: {len(val_dataset)}')
    
    # Data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=config['num_workers'],
        pin_memory=True,
        drop_last=True  # For stable batch norm
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=config['num_workers'],
        pin_memory=True
    )
    
    # Train model
    model, history = train_canet(
        train_loader,
        val_loader,
        epochs=config['epochs'],
        lr=config['lr'],
        weight_decay=config['weight_decay'],
        lambda_aux=config['lambda_aux'],
        device=device
    )
    
    print('\nTraining completed successfully!')


if __name__ == '__main__':
    main()

Using device: cuda
Train samples: 413
Val samples: 103
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 228MB/s]


Total parameters: 26.16M
Trainable parameters: 26.16M

Epoch 1/100


Validation: 100%|██████████| 7/7 [00:07<00:00,  1.04s/it, loss=2.8566]



Learning Rate: 0.000098

Train Loss: 2.8197
  - DR Main: 1.4235
  - DME Main: 0.8409
  - DR Aux: 1.4132
  - DME Aux: 0.8080

Validation Loss: 2.8566

DR Metrics:
  accuracy: 0.4563
  auc: 0.6350
  precision: 0.1784
  recall: 0.2824
  f1_score: 0.2167
  sensitivity: 0.5362
  specificity: 0.9118

DME Metrics:
  accuracy: 0.7864
  auc: 0.8546
  precision: 0.5277
  recall: 0.5815
  f1_score: 0.5513
  sensitivity: 0.7414
  specificity: 0.9111

Joint Accuracy: 0.4563

✓ Saved best joint accuracy model: 0.4563
✓ Saved best DR AUC model: 0.6350
Saved best DME AUC model: 0.8546

Epoch 2/100


...


Learning Rate: 0.000073

Train Loss: 0.3349
  - DR Main: 0.2026
  - DME Main: 0.0635
  - DR Aux: 0.2035
  - DME Aux: 0.0716

Validation Loss: 5.5225

DR Metrics:
  accuracy: 0.4951
  auc: 0.7570
  precision: 0.4343
  recall: 0.3678
  f1_score: 0.3667
  sensitivity: 0.7536
  specificity: 0.6176

DME Metrics:
  accuracy: 0.7573
  auc: 0.8496
  precision: 0.5227
  recall: 0.5593
  f1_score: 0.5392
  sensitivity: 0.7414
  specificity: 0.8444

Joint Accuracy: 0.4466

Epoch 99/100


Validation: 100%|██████████| 7/7 [00:07<00:00,  1.05s/it, loss=4.9338]



Learning Rate: 0.000071

Train Loss: 0.4376
  - DR Main: 0.2405
  - DME Main: 0.1095
  - DR Aux: 0.2314
  - DME Aux: 0.1189

Validation Loss: 4.9338

DR Metrics:
  accuracy: 0.5146
  auc: 0.7809
  precision: 0.4135
  recall: 0.3854
  f1_score: 0.3892
  sensitivity: 0.8406
  specificity: 0.6471

DME Metrics:
  accuracy: 0.7767
  auc: 0.8531
  precision: 0.6627
  recall: 0.6528
  f1_score: 0.6548
  sensitivity: 0.7759
  specificity: 0.8667

Joint Accuracy: 0.4466

Epoch 100/100


Validation: 100%|██████████| 7/7 [00:07<00:00,  1.02s/it, loss=5.1324]


Learning Rate: 0.000069

Train Loss: 0.3270
  - DR Main: 0.1640
  - DME Main: 0.0869
  - DR Aux: 0.1873
  - DME Aux: 0.1173

Validation Loss: 5.1324

DR Metrics:
  accuracy: 0.4757
  auc: 0.7722
  precision: 0.4275
  recall: 0.4245
  f1_score: 0.4104
  sensitivity: 0.8406
  specificity: 0.5294

DME Metrics:
  accuracy: 0.7670
  auc: 0.8554
  precision: 0.5963
  recall: 0.5931
  f1_score: 0.5896
  sensitivity: 0.7586
  specificity: 0.8667

Joint Accuracy: 0.3981

Training completed!
Best Joint Accuracy: 0.5922
Best DR AUC: 0.8278
Best DME AUC: 0.9032

Training completed successfully!
